# I2B2


In [14]:
# %load_ext autoreload

# %autoreload

import os, re
import pandas as pd
from const import SEEDS, MARKERS_I2B2, MODELS, CONTEXT_WINDOWS, LRS, SEEDS, MARKERS_TBD, TRAIN_BATCH_SIZE

task="i2b2"
num_train_epochs=10
i2b2_dataset="i2b2_data/relations"
test_xml_dir = "preprocess/corpus/i2b2/ground_truth/merged_xml"
train_batch_size = 32

BERT_ONLY=False


if BERT_ONLY:
   relation_type = ['relations_plain_bert']
else:
    relation_type = ['relations_more_link_with_so'] 
  
MARKERS_I2B2=["ner"]

CONTEXT_WINDOWS=[2]

SEEDS=[1, 2 ,3 ,4 ,5]

res_sub_df_dict = {}

res_link_type_df_dict = {}
res_doc_dict = {}
res_marker_df_dict = {}

for marker in MARKERS_I2B2:
    for model in MODELS:
        for context_window in CONTEXT_WINDOWS:
            for lr in LRS:
                for train_batch_size in TRAIN_BATCH_SIZE:
                    for rel_type in relation_type:
                        for seed in SEEDS:
                            slurm_out_name = f"{marker}_{model}_{context_window}_{lr}_{num_train_epochs}_{train_batch_size}_{seed}"
                            dict_out_name = f"{marker}_{model}_{context_window}_{lr}_{num_train_epochs}_{train_batch_size}"
                            i2b2_rel_model = f"i2b2_models/{rel_type}/{marker}"
                            output_dir = f"{i2b2_rel_model}/{slurm_out_name}"
                            
                            
                            result_file = os.path.join(output_dir, "result_test.txt")
                            result_file_sub = os.path.join(output_dir, "result_sub.csv")
                            result_file_link_type = os.path.join(output_dir, "result_link_type.csv")
                            result_file_marker_type = os.path.join(output_dir, "result_marker_type.csv")

                            if os.path.exists(result_file):
                                print('result...')
                                with open(result_file, 'r') as f:
                                    lines = f.readlines()
                                    res_idx = 6
                                    #print(lines[-res_idx:])
                                    # res = lines[-res_idx:]
                                    res_keys = ['Precicion', 'Recall', 'Average P&R', 'F measure']
                                    res_dict = {}
                                    
                                    for i, (k, line)in enumerate(zip(res_keys, lines[-res_idx:])):
                                        if k!='Average P&R':
                                            m = re.search(r".*\t(.*)\n.*", line)
                                            val = m.group(1)
                                            res_dict[k] = val
                                    res_dict['seed'] = seed
                                    res_dict['method'] = dict_out_name
                                    res_dict['rel_type'] = rel_type
                                    res_doc_dict[(slurm_out_name, rel_type)] = res_dict
                            else:

                                import pdb
                                pdb.set_trace()

                            if os.path.exists(result_file_sub):
                                res_sub_df_dict[(dict_out_name, seed, rel_type)] = pd.read_csv(result_file_sub)
                            if os.path.exists(result_file_link_type):    
                                res_link_type_df_dict[(dict_out_name, seed, rel_type)] = pd.read_csv(result_file_link_type)
                            if os.path.exists(result_file_marker_type):
                                res_marker_df_dict[(dict_out_name, seed, rel_type)] = pd.read_csv(result_file_marker_type, index_col=0)
                                
                                
df = pd.DataFrame.from_dict(res_doc_dict, orient="index")       
df
vals = ['Precicion', 'Recall', 'F measure', 'seed', 'rel_type']                   
for v in vals:
    if v!= 'rel_type':
        
        df[v] = pd.to_numeric(df[v])

df_grouped = df.groupby(['method', 'rel_type']).agg(['mean', 'std']).sort_values(by=[('F measure', 'mean'), 'method'])


df_grouped = df_grouped.reset_index()

df_grouped

result...
result...
result...
result...
result...


method                     rel_type Precicion  \
                                                                        mean   
0  ner_bert-base-uncased_2_2e-5_10_16  relations_more_link_with_so   0.82988   

              Recall           F measure           seed            
        std     mean       std      mean       std mean       std  
0  0.006255  0.83918  0.004427   0.83452  0.005354  3.0  1.581139

In [13]:

df_grouped = df_grouped.pivot(index='method', columns='rel_type')#, values=['rel_type', 'F measure', 'Precision'])#.sort_values(by=[('F measure', 'mean', 'relations_more_link_with_so')])
df_grouped
#df_grouped[[('Precicion', 'mean', 'relations_plain_bert')]]


Precicion  \
                                                          mean   
rel_type                           relations_more_link_with_so   
method                                                           
ner_bert-base-uncased_2_2e-5_10_16                     0.82988   

                                                                \
                                                           std   
rel_type                           relations_more_link_with_so   
method                                                           
ner_bert-base-uncased_2_2e-5_10_16                    0.006255   

                                                        Recall  \
                                                          mean   
rel_type                           relations_more_link_with_so   
method                                                           
ner_bert-base-uncased_2_2e-5_10_16                     0.83918   

                                                                \
                                                           std   
rel_type                           relations_more_link_with_so   
method                                                           
ner_bert-base-uncased_2_2e-5_10_16                    0.004427   

                                                     F measure  \
                                                          mean   
rel_type                           relations_more_link_with_so   
method                                                           
ner_bert-base-uncased_2_2e-5_10_16                     0.83452   

                                                                \
                                                           std   
rel_type                           relations_more_link_with_so   
method                                                           
ner_bert-base-uncased_2_2e-5_10_16                    0.005354   

                                                          seed  \
                                                          mean   
rel_type                           relations_more_link_with_so   
method                                                           
ner_bert-base-uncased_2_2e-5_10_16                         3.0   

                                                                
                                                           std  
rel_type                           relations_more_link_with_so  
method                                                          
ner_bert-base-uncased_2_2e-5_10_16                    1.581139

# sub group analysis of optimal method

In [ ]:
len(res_sub_df_dict)

5

In [ ]:
from functools import reduce

optimal = 'ner_bert-base-uncased_2_2e-5_10_16'

vals_rename = {'precision':'P', 'recall':'R', 'f1':'F1'}

output_names = vals_rename.values()

lst_of_optimal_df = []
rel_type='relations_more_link_with_so'

for (out_file, seed, relation_type) in res_sub_df_dict.keys():
    if out_file==optimal and relation_type==rel_type:
        df_format = res_sub_df_dict[(optimal, seed, relation_type)].rename(columns=vals_rename)[output_names].set_axis(['ALL', 'BEFORE', 'AFTER', 'OVERLAP'])#['ner_pos_bert-base-uncased_1_2e-5_10_32']
        lst_of_optimal_df.append(df_format)
#pd.DataFrame.from_dict(res_sub_df_dict)

# 'test': {'BEFORE': 15113, 'AFTER': 2729, 'OVERLAP': 9893}}
label_dist = [2729,  9893, 15113]
total = sum(label_dist)
def f1(x):
    return f"{x*100:.1f}"
def f_dis(x, total=total):
    proportion = (x/total)*100
    return f"{x:,} ({proportion:.1f})"


    


df_final = reduce(lambda x, y: x.add(y, fill_value=0), lst_of_optimal_df) / len(lst_of_optimal_df)
df_final = df_final.iloc[[2, 3, 1, 0]]
df_final['Count (percentage)'] = label_dist + [total]
df_final = df_final[['Count (percentage)', 'P', 'R', 'F1']]

print(df_final.to_latex(formatters=[f_dis] + [f1 for i in range(1, 4)]))

\begin{tabular}{lrrrr}
\toprule
{} & Count (percentage) &    P &    R &   F1 \\
\midrule
AFTER   &        2,729 (9.8) & 67.3 & 65.4 & 66.3 \\
OVERLAP &       9,893 (35.7) & 82.9 & 88.6 & 85.7 \\
BEFORE  &      15,113 (54.5) & 88.7 & 83.6 & 86.1 \\
ALL     &     27,735 (100.0) & 83.0 & 83.9 & 83.4 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_12105/1379149608.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_final.to_latex(formatters=[f_dis] + [f1 for i in range(1, 4)]))


# Link type analysis

In [ ]:
res_link_type_df_dict


{('ner_bert-base-uncased_2_2e-5_10_16',
  1,
  'relations_more_link_with_so'):    precision    recall        pr        f1
 0   0.819703  0.831770  0.825657  0.825692
 1   0.822169  0.872253  0.845821  0.846471
 2   0.815570  0.817048  0.816310  0.816308
 3   0.819915  0.851971  0.835371  0.835635
 4   0.721154  0.719807  0.720482  0.720480
 5   0.821360  0.822263  0.821812  0.821811,
 ('ner_bert-base-uncased_2_2e-5_10_16',
  2,
  'relations_more_link_with_so'):    precision    recall        pr        f1
 0   0.836731  0.843347  0.840011  0.840026
 1   0.860831  0.886018  0.872964  0.873243
 2   0.835548  0.838669  0.837108  0.837106
 3   0.851960  0.868622  0.860094  0.860210
 4   0.711538  0.714976  0.713253  0.713253
 5   0.829889  0.831241  0.830565  0.830565,
 ('ner_bert-base-uncased_2_2e-5_10_16',
  3,
  'relations_more_link_with_so'):    precision    recall        pr        f1
 0   0.830033  0.841262  0.835600  0.835610
 1   0.843400  0.889399  0.865521  0.865789
 2   0.826594  0

In [ ]:
res_link_type_df_dict




{('ner_bert-base-uncased_2_2e-5_10_16',
  1,
  'relations_more_link_with_so'):    precision    recall        pr        f1
 0   0.819703  0.831770  0.825657  0.825692
 1   0.822169  0.872253  0.845821  0.846471
 2   0.815570  0.817048  0.816310  0.816308
 3   0.819915  0.851971  0.835371  0.835635
 4   0.721154  0.719807  0.720482  0.720480
 5   0.821360  0.822263  0.821812  0.821811,
 ('ner_bert-base-uncased_2_2e-5_10_16',
  2,
  'relations_more_link_with_so'):    precision    recall        pr        f1
 0   0.836731  0.843347  0.840011  0.840026
 1   0.860831  0.886018  0.872964  0.873243
 2   0.835548  0.838669  0.837108  0.837106
 3   0.851960  0.868622  0.860094  0.860210
 4   0.711538  0.714976  0.713253  0.713253
 5   0.829889  0.831241  0.830565  0.830565,
 ('ner_bert-base-uncased_2_2e-5_10_16',
  3,
  'relations_more_link_with_so'):    precision    recall        pr        f1
 0   0.830033  0.841262  0.835600  0.835610
 1   0.843400  0.889399  0.865521  0.865789
 2   0.826594  0

In [ ]:
res_link_type_df_dict


output_names = vals_rename.values()

lst_of_optimal_df = []
link_types = ['ALL', 'SEC', 'ET_OTHER', 'ET', 'TT', 'EE']
for (out_file, seed, rel_trunc_type) in res_link_type_df_dict.keys():
    if out_file==optimal: #and rel_trunc_type=="relations_truncate":
        df_format = res_link_type_df_dict[(optimal, seed, rel_trunc_type)].rename(columns=vals_rename)[output_names].set_axis(link_types)#['ner_pos_bert-base-uncased_1_2e-5_10_32']
        lst_of_optimal_df.append(df_format)
#pd.DataFrame.from_dict(res_link_type_df_dict)

# label_dist = [2725,  9888, 15041]

def f1(x):
    return f"{x*100:.1f}"
def f_dis(x, total=total):
    proportion = (x/total)*100
    return f"{x:,} ({proportion:.1f})"
    
link_type_count = pd.read_csv(os.path.join(f"/data/chengc7/MarkerTRel/i2b2_data/{relation_type}", "test_link_count_by_type.csv"))
df_final = reduce(lambda x, y: x.add(y, fill_value=0), lst_of_optimal_df) / len(lst_of_optimal_df)
link_type_count = link_type_count.iloc[[0, 1, 3, 4]]
total = link_type_count['0'].values.sum()

df_final = df_final[~df_final.index.isin(["ET", "ALL"])]
df_final['Count'] = link_type_count['0'].values
df_final = df_final[['Count', 'P', 'R', 'F1']]

vals_order = ['']
df_final
print(df_final.to_latex(formatters=[f_dis] + [f1 for i in range(1, 4)]))

\begin{tabular}{lrrrr}
\toprule
{} &         Count &    P &    R &   F1 \\
\midrule
SEC      & 13,317 (48.0) & 84.7 & 88.5 & 86.5 \\
ET\_OTHER &   2,452 (8.8) & 82.6 & 82.8 & 82.7 \\
TT       &     214 (0.8) & 73.3 & 73.6 & 73.4 \\
EE       & 11,752 (42.4) & 82.6 & 82.7 & 82.6 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_12105/2337884955.py:37: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_final.to_latex(formatters=[f_dis] + [f1 for i in range(1, 4)]))


# TBD

In [23]:
%load_ext autoreload
%autoreload
import sys
import pandas as pd
from IPython.display import clear_output

# run python aggregate_tbd.py first 
df = pd.read_csv("agg_tbd.csv")
df.sort_values(by='f1')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,method,precision,recall,f1,seed
0,pos_bert-base-uncased_1_2e-5_20_8_1,0.592151,0.592151,0.592151,1.0
1,pos_tense_polarity_bert-base-uncased_0_2e-5_20...,0.629292,0.629292,0.629292,1.0
